# Notebook of how linear regression can be added to the main code #

In [12]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import plotly
import plotly.express as px
import plotly.graph_objs as go

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import os

In [2]:
GEO_LSOA_PATH = os.getcwd() + '/data/Geo_data/LSOA_2011_London_gen_MHW.shp'
TESCO_PATH = os.getcwd() + '/data/Area_level_data/year_lsoa_grocery.csv'
SOCIO_ECO_LSOA_PATH = os.getcwd() + '/data/lsoa-data.csv'

In [3]:
def load_merge_clean_data(TESCO_PATH, SOCIO_ECO_LSOA_PATH, GEO_LSOA_PATH):
    '''
    This function load the different datasets used for the analysis,
    clean and merge those datasets together to obtain as an output one single panda dataframe
    with the socio-economic, the Tesco and the geography information of each LSOA
    '''
    # load the data with the geography information of each LSOA
    map_df = gpd.read_file(GEO_LSOA_PATH)
    # set the index of this dataframe to the code of each LSOA to facilitate the merge of the dataframes
    map_df.index = map_df['LSOA11CD']
    
    # load the data with the Tesco information of each LSOA
    data_df = pd.read_csv(TESCO_PATH)
    
    # merge the Tesco dataframe with the one with the geo information of the corresponding regions  
    merged_map_df = map_df.join(data_df.set_index('area_id'))
    
    # convert coordinates that are in UTM format into latitude longitude (to plot the results on a map)
    merged_map_df = merged_map_df.to_crs({'init': 'epsg:4326'}) # cause of the warning
    
    # load the data with the socio-economic information of each LSOA
    lsoa_df = pd.read_csv(SOCIO_ECO_LSOA_PATH, encoding = "ISO-8859-1", engine='python')
    # drop the last 2 rows that are full of nan (due to the format of the title of the columns (3 rows))
    lsoa_df.drop(lsoa_df.tail(2).index,inplace=True)
    
    # merge the merged dataframe with  the socio-economic dataframe of the corresponding regions  
    merged_map_lsoa_df = merged_map_df.join(lsoa_df.set_index('Lower Super Output Area'))
    
    # set the index of this dataframe to the name of each LSOA to obtain more comprehensive data
    merged_map_lsoa_df.index = merged_map_lsoa_df['LSOA11NM']
    
    # remove the rows for which we don't have data everywhere 
    # consider the feature of population to determine where we lack some information
    population = np.array((merged_map_lsoa_df['population'])) 
    merged_map_lsoa_df = merged_map_lsoa_df[np.logical_not(np.isnan(population))]
    
    # return this final merged and cleaned dataset
    return merged_map_lsoa_df

In [4]:
df = load_merge_clean_data(TESCO_PATH, SOCIO_ECO_LSOA_PATH, GEO_LSOA_PATH)

In [5]:
df.head()

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,...,Road Casualties;2012;Slight,Road Casualties;2012;2012 Total,Road Casualties;2013;Fatal,Road Casualties;2013;Serious,Road Casualties;2013;Slight,Road Casualties;2013;2013 Total,Road Casualties;2014;Fatal,Road Casualties;2014;Serious,Road Casualties;2014;Slight,Road Casualties;2014;2014 Total
LSOA11NM,,,,,,,,,,,,,,,,,,,,,
City of London 001A,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,1465,1465,...,14.0,16.0,0.0,3.0,10.0,13.0,0.0,2.0,10.0,12.0
City of London 001B,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,1436,1436,...,8.0,9.0,0.0,1.0,5.0,6.0,0.0,0.0,9.0,9.0
City of London 001C,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London,1346,1250,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
City of London 001E,E01000005,City of London 001E,E02000001,City of London 001,E09000001,City of London,E12000007,London,985,985,...,22.0,24.0,0.0,5.0,15.0,20.0,1.0,0.0,20.0,21.0
Barking and Dagenham 016A,E01000006,Barking and Dagenham 016A,E02000017,Barking and Dagenham 016,E09000002,Barking and Dagenham,E12000007,London,1703,1699,...,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,3.0,3.0


## Standardize the features of interest ##

In [6]:
eco_activ = df['Economic Activity;Employment Rate;2011']
eco_activ_stand = (eco_activ - np.mean(eco_activ))/np.std(eco_activ)

income = df['Household Income, 2011/12;Median Annual Household Income estimate (£)']
income_stand = (income - np.mean(income))/np.std(income)

age = df['avg_age']
age_stand = (age - np.mean(age))/np.std(age)

h_items = df['h_items_norm']
h_items_stand = (h_items - np.mean(h_items))/np.std(h_items)

In [7]:
diversity_feats = ['Ethnic Group;Mixed/multiple ethnic groups (%);2011', 
                   'Ethnic Group;Asian/Asian British (%);2011', 
                   'Ethnic Group;Black/African/Caribbean/Black British (%);2011', 
                   'Ethnic Group;Other ethnic group (%);2011']
h_ethnicity = 0
for feat in diversity_feats:
    val_feat = df[feat]/100
    h_ethnicity += -val_feat * np.log2(val_feat+0.001)
h_ethnicity = h_ethnicity/np.log2(4)

h_ethnicity_stand = (h_ethnicity - np.mean(h_ethnicity))/np.std(h_ethnicity)

In [10]:
stand_df = pd.DataFrame()
stand_df['eco'] = eco_activ_stand
stand_df['income'] = income_stand
stand_df['age'] = age_stand
stand_df['h_items'] = h_items_stand
stand_df['h_ethnicity'] = h_ethnicity_stand
#stand_df['h_ethni_bin'] = [1 if eth > np.median(h_ethnicity) else 0 for eth in h_ethnicity]
stand_df.head()

,eco,income,age,h_items,h_ethnicity
LSOA11NM,,,,,
City of London 001A,1.465458,1.971130,3.214820,-1.382446,-1.344442
City of London 001B,1.453738,1.867475,2.976434,-1.104701,-1.900963
City of London 001C,0.563031,-0.119432,3.232598,0.359911,-0.762202
City of London 001E,-0.409714,-1.025721,-0.162352,-0.182977,0.932069
Barking and Dagenham 016A,-0.444873,0.295365,-1.102792,-0.245228,0.398236


## Predicting food consumption diversity using linear regression ##

First, we will try to keep only the ethnic entropy (`h_ethnicity`) to try to explain the food consumption entropy (`items`).

In [13]:
train_df, test_df = train_test_split(stand_df, test_size=0.3, random_state=42)

feature_cols = ['h_ethnicity']

X_train = train_df[feature_cols]
X_test = test_df[feature_cols]
y_train = train_df['h_items']
y_test = test_df['h_items']

lin_reg = LinearRegression()  # create the model
lin_reg.fit(X_train, y_train)  # train
y_pred_test = lin_reg.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)

print('The ethnic entropy explains {0:.2f} % of the variance of the food consumption entropy.'
      .format(r2_test*100))

The ethnic entropy explains 12.90 % of the variance of the food consumption entropy.


We will now try to explain the variance in the food consumption entropy (`h_items`) with the following features of each LSOA:
- The ethnic entropy: `h_ethnicity`
- The employment rate: `eco`
- The average age: `age`
- The average income per household: `income`

In [14]:
feature_cols = ['eco', 'income', 'age', 'h_ethnicity']

# Linear regression with random split
X_train = train_df[feature_cols]
X_test = test_df[feature_cols]
y_train = train_df['h_items']
y_test = test_df['h_items']

lin_reg = LinearRegression()  # create the model
lin_reg.fit(X_train, y_train)  # train
y_pred_test = lin_reg.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)

print('The feature columns used here explain {0:.2f} % of the variance of the food consumption entropy.'
      .format(r2_test*100))

The feature columns used here explain 19.88 % of the variance of the food consumption entropy.


When adding the employment rate, the median household income and the average age of the LSOA's population in the predicting factors, $R^2$ only rises from 12.9 to 19.9 \%. This seems to suggest that the ethnic entropy has a non-negligible effect on the food category entropy.

We will now use the statsmodels module, and summarize the results to see the contribution and significance of each term. 

In [15]:
formula = 'h_items ~ h_ethnicity + eco + age + income'

mod = smf.ols(formula=formula, data=stand_df)
np.random.seed(2)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                h_items   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     276.2
Date:                Tue, 08 Dec 2020   Prob (F-statistic):          4.11e-214
Time:                        22:02:35   Log-Likelihood:                -6359.8
No. Observations:                4833   AIC:                         1.273e+04
Df Residuals:                    4828   BIC:                         1.276e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    1.323e-15      0.013   1.02e-13      

From these results, it looks like all four predictive factors have a significant effect on the food category entropy. They suggest the following conclusions:
- LSOA with more ethnic diversity tend to have a larger food consumption diversity
- LSOA with a higher employment rate tend to have a larger food consumption diversity
- LSOA with a higher average age tend to have a lower food consumption diversity
- LSOA with a larger median income per household tend to have a lower food consumption diversity

As the features are standardized, the magnitude of their coefficients can be directly compared. This suggests that `income` impacts `h_items` the most, followed by `h_ethnicity`, `eco` and finally `age`. Even though it seems that ethnic diversity has an effect on food consumption diversity in an LSOA, we still need to be careful with the conclusions we might take here. Indeed, we could be in a situation where for example the `income` feature is a confounder and therefore it influences both `h_items` and `h_ethnicity`. We could therefore see an effect of `h_ethnicity` on `h_items`, even though it is actually `income` which impacts both indirectly. In this case, the effect of `h_ethnicity` on `h_items` could entirely be explained by `income`. To ensure that we are not in such a situation, we will use the notion of matching. We will first create a binary feature from `h_ethnicity` to divide the dataset in two groups. We will then perform a matching between these two groups, using the concept of propensity score. This will allow us to compare similar sets of samples and to get more insight on the effect of the ethnic diversity on the food consumption diversity.